In [1]:
import nibabel as nib
from totalsegmentator.python_api import totalsegmentator
import os
import SimpleITK as sitk
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
from image_utils import convert_series_to_nifti, quantize_maps
import registration_gui as rgui
import logging
from image_utils import ImageVisualizer, delete_niigz_files
%matplotlib inline

In [5]:
transforms_dir = "D:/CTH_archive/TRANSFORMS"
fixed_images_dir = "D:/CTH_archive/CTH_STRIPPED"  
moving_images_dir = "D:/CTH_archive/CTP_STRIPPED" 

def register_images(fixed_image_path, moving_image_path, transforms_dir):
    # Extract patient identifier from the file name, ensuring .nii is not included
    patient = os.path.splitext(os.path.basename(moving_image_path))[0]
    patient = os.path.splitext(patient)[0]  # Remove .nii if present

    # Construct the transform file path
    transform_file = os.path.join(transforms_dir, f'{patient}.h5')

    # Check if the transform file already exists and skip registration if it does
    if os.path.exists(transform_file):
        print(f"Transform file already exists for patient {patient}, skipping registration.")
        return

    # Load the fixed and moving images
    fixed_image = sitk.ReadImage(fixed_image_path)
    moving_image = sitk.ReadImage(moving_image_path)

    
    initial_transform = sitk.CenteredTransformInitializer(
    fixed_image,
    moving_image,
    sitk.Euler3DTransform(),
    sitk.CenteredTransformInitializerFilter.GEOMETRY,
)
    
    registration_method = sitk.ImageRegistrationMethod()

    # Similarity metric settings.
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.10)

    registration_method.SetInterpolator(sitk.sitkNearestNeighbor)

    # Optimizer settings.
    registration_method.SetOptimizerAsGradientDescent(
        learningRate=1.0,
        numberOfIterations=1000,
        convergenceMinimumValue=1e-6,
        convergenceWindowSize=10,
    )
    registration_method.SetOptimizerScalesFromPhysicalShift()

    # Setup for the multi-resolution framework.
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors=[4, 2, 1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2, 1, 0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    # Set the initial moving and optimized transforms.
    optimized_transform = sitk.Euler3DTransform()
    registration_method.SetMovingInitialTransform(initial_transform)
    registration_method.SetInitialTransform(optimized_transform, inPlace=False)

    # Connect all of the observers so that we can perform plotting during registration.
    #registration_method.AddCommand(sitk.sitkStartEvent, rgui.start_plot)
    #registration_method.AddCommand(sitk.sitkEndEvent, rgui.end_plot)
    #registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, rgui.update_multires_iterations)
    #registration_method.AddCommand(sitk.sitkIterationEvent, lambda: rgui.plot_values(registration_method))

    try:
        # Execute the registration
        registration_method.Execute(fixed_image, moving_image)

        # Always check the reason optimization terminated.
        print("Final metric value: {0}".format(registration_method.GetMetricValue()))
        print("Optimizer's stopping condition, {0}".format(registration_method.GetOptimizerStopConditionDescription()))
        
        # Instead of creating and saving a CompositeTransform, save the optimized_transform directly
        sitk.WriteTransform(optimized_transform, transform_file)

        # If you need to apply the initial transform as well, consider applying it to the moving image first and then register the result to the fixed image.
        # Alternatively, you can apply both transforms to the moving image for resampling, without creating a CompositeTransform.

        # Resample the moving image using the final (optimized) transform
        resampled_moving_image = sitk.Resample(moving_image, fixed_image, optimized_transform, sitk.sitkNearestNeighbor)
        sitk.WriteImage(resampled_moving_image, f"D:/CTH_archive/CTP_STRIPPED_REG/{patient}.nii")

        print(f"Registration successful for patient: {patient}. Transform saved to {transform_file}")
    except RuntimeError as e:
        print(f"Registration failed for patient {patient}: {e}")

for filename in tqdm(os.listdir(fixed_images_dir)):
    fixed_image_path = os.path.join(fixed_images_dir, filename)
    moving_image_path = os.path.join(moving_images_dir, filename)

    if os.path.isfile(fixed_image_path) and os.path.isfile(moving_image_path):
        register_images(fixed_image_path, moving_image_path, transforms_dir)


  0%|          | 0/107 [00:00<?, ?it/s]

Transform file already exists for patient ALFORD_BARBARA, skipping registration.
Transform file already exists for patient ALLAH_MAJUSTICE, skipping registration.
Transform file already exists for patient BATTLE_MARIA, skipping registration.
Transform file already exists for patient BAUM_ROBERT, skipping registration.
Transform file already exists for patient BILLIPS_JAMES, skipping registration.
Transform file already exists for patient BOGER_DAVID_S, skipping registration.
Transform file already exists for patient BROWN_ANTHONY, skipping registration.
Transform file already exists for patient CAMPAGNA_HARRY_D, skipping registration.
Transform file already exists for patient CANIGLIA_ROBERT, skipping registration.
Transform file already exists for patient CARDIN_PAUL, skipping registration.
Transform file already exists for patient CHANG_WAH_KONG, skipping registration.
Transform file already exists for patient CHEN_QIAOYING, skipping registration.
Transform file already exists for pa

In [6]:
FINAL_SIZE = 512

def resample_image(moving_image, ctp_image):
    desired_size = [FINAL_SIZE, FINAL_SIZE, ctp_image.GetSize()[2]] # Use the same number of slices as the CTP image
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(moving_image)
    resampler.SetSize(desired_size)
    resampler.SetOutputSpacing([moving_image.GetSpacing()[i] * (moving_image.GetSize()[i] / desired_size[i]) for i in range(3)]) 
    resampler.SetInterpolator(sitk.sitkLinear)
    resized_moving_image = resampler.Execute(moving_image)
    resized_moving_image.SetSpacing(ctp_image.GetSpacing())
    resized_moving_image.SetOrigin(ctp_image.GetOrigin())
    resized_moving_image.SetDirection(ctp_image.GetDirection())
    return resized_moving_image

def apply_final_transform(resized_moving_image, fixed_image, transform_file_path):
    final_transform = sitk.ReadTransform(transform_file_path)
    resampled_image = sitk.Resample(resized_moving_image, 
                                    fixed_image, 
                                    final_transform, 
                                    sitk.sitkNearestNeighbor, 
                                    0.0, 
                                    fixed_image.GetPixelID())
    return resampled_image

def apply_final_transform(resized_moving_image, fixed_image, transform_file_path):
    try:
        final_transform = sitk.ReadTransform(transform_file_path)
        
        # Ensure the resized_moving_image is valid before proceeding
        if resized_moving_image:
            resampled_image = sitk.Resample(resized_moving_image, 
                                            fixed_image, 
                                            final_transform, 
                                            sitk.sitkLinear, 
                                            0.0, 
                                            fixed_image.GetPixelID())
            return resampled_image
        else:
            logging.error("Resized moving image is invalid. Cannot apply final transform.")
            return None
    except Exception as e:
        logging.error("Failed to apply final transform: " + str(e))
        return None

tmax_nifti_dir = r"D:/CTH_archive/TMAX_NIFTI"
transforms_dir = r"D:/CTH_archive/TRANSFORMS_TEST"
cth_stripped_dir = r"D:/CTH_archive/CTH_STRIPPED"
ctp_stripped_dir = r"D:/CTH_archive/CTP_STRIPPED"
output_dir = r'D:/CTH_archive/TMAX_REGISTERED_TEST'

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate over the transformation files
for transform_file in tqdm(os.listdir(transforms_dir)):
    transform_path = os.path.join(transforms_dir, transform_file)
    base_filename = transform_file.replace('.h5', '')

    # Paths to the moving, fixed, and reference (ctp) images
    moving_image_path = os.path.join(tmax_nifti_dir, base_filename + '.nii')
    fixed_image_path = os.path.join(cth_stripped_dir, base_filename + '.nii')
    ctp_image_path = os.path.join(ctp_stripped_dir, base_filename + '.nii')
    print(f"Processing {base_filename}...")

    # Check if all required files exist
    if os.path.exists(moving_image_path) and os.path.exists(fixed_image_path) and os.path.exists(ctp_image_path):
        # Load images
        moving_image = sitk.ReadImage(moving_image_path)
        fixed_image = sitk.ReadImage(fixed_image_path)
        ctp_image = sitk.ReadImage(ctp_image_path)

        # Resample the moving image
        resized_moving_image = resample_image(moving_image, ctp_image)

        # Apply the final transformation
        resampled_image = apply_final_transform(resized_moving_image, fixed_image, transform_path)

        # Save the resampled image
        resampled_image_path = os.path.join(output_dir, base_filename + '.nii')
        sitk.WriteImage(resampled_image, resampled_image_path)
        print(f"Processed and saved: {resampled_image_path}")
    else:
        print(f"Required files for {base_filename} are not available.")

Processing ALFORD_BARBARA...
Processed and saved: D:/CTH_archive/TMAX_REGISTERED_TEST\ALFORD_BARBARA.nii
Processing ALLAH_MAJUSTICE...
Processed and saved: D:/CTH_archive/TMAX_REGISTERED_TEST\ALLAH_MAJUSTICE.nii
Processing BATTLE_MARIA...
Processed and saved: D:/CTH_archive/TMAX_REGISTERED_TEST\BATTLE_MARIA.nii
Processing BAUM_ROBERT...
Processed and saved: D:/CTH_archive/TMAX_REGISTERED_TEST\BAUM_ROBERT.nii
Processing BILLIPS_JAMES...
Processed and saved: D:/CTH_archive/TMAX_REGISTERED_TEST\BILLIPS_JAMES.nii
Processing BOGER_DAVID_S...
Processed and saved: D:/CTH_archive/TMAX_REGISTERED_TEST\BOGER_DAVID_S.nii
Processing BROWN_ANTHONY...
Processed and saved: D:/CTH_archive/TMAX_REGISTERED_TEST\BROWN_ANTHONY.nii
Processing CAMPAGNA_HARRY_D...
Processed and saved: D:/CTH_archive/TMAX_REGISTERED_TEST\CAMPAGNA_HARRY_D.nii
Processing CANIGLIA_ROBERT...
Processed and saved: D:/CTH_archive/TMAX_REGISTERED_TEST\CANIGLIA_ROBERT.nii
Processing CARDIN_PAUL...
Processed and saved: D:/CTH_archive/T